In [13]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd
from collections import Counter
import re
import pickle
import MeCab
mc = MeCab.Tagger("-Ochasen")

In [14]:
label_excel = pd.ExcelFile("data/excel2/new_label_2.xlsx")
sheet_name = label_excel.sheet_names[0]
label_df = label_excel.parse(sheet_name)

In [15]:
label_df

,id,reply,group_id,cname,body,Espistemic,Argumentation,Coordination,Social
0,10730,0,1,飼育係,よろしくー\n\n,off task,non-argumentative moves,blank,quick consensus building
1,10749,0,1,ドラ,よろしくお願いします～,off task,non-argumentative moves,blank,externalization
2,10754,0,1,海の日,よろしくお願いしまーす,off task,non-argumentative moves,blank,externalization
3,10849,0,1,飼育係,なんかやり方違うｗｗｗｗｗｗｗｗｗｗｗｗｗｗ,on task,non-argumentative moves,blank,externalization
4,10903,0,1,飼育係,拝見しましたが特に問題はないです,on task,non-argumentative moves,blank,quick consensus building
5,10908,0,1,ドラ,とりあえずこのまま進めてみますか？,on task,non-argumentative moves,blank,elicitation
6,10923,0,1,飼育係,そうしてくれると助かります,off task,non-argumentative moves,blank,externalization
7,11065,0,1,ドラ,お二人のワード拝見しました。\n\n内容については特に疑問点はありませんでした。,on task,non-argumentative moves,blank,quick consensus building
8,11070,0,1,ドラ,お二人のワード拝見しました。\n\n内容については特に疑問点はありませんでした。,on task,non-argumentative moves,blank,quick consensus building
9,11135,0,1,海の日,私も特にありませんでした。,on task,non-argumentative moves,technical coordination,externalization


In [19]:
label_df_1 = label_df.drop(['id','reply','body'],axis=1)
label_df_1

,group_id,cname,Espistemic,Argumentation,Coordination,Social
0,1,飼育係,off task,non-argumentative moves,blank,quick consensus building
1,1,ドラ,off task,non-argumentative moves,blank,externalization
2,1,海の日,off task,non-argumentative moves,blank,externalization
3,1,飼育係,on task,non-argumentative moves,blank,externalization
4,1,飼育係,on task,non-argumentative moves,blank,quick consensus building
5,1,ドラ,on task,non-argumentative moves,blank,elicitation
6,1,飼育係,off task,non-argumentative moves,blank,externalization
7,1,ドラ,on task,non-argumentative moves,blank,quick consensus building
8,1,ドラ,on task,non-argumentative moves,blank,quick consensus building
9,1,海の日,on task,non-argumentative moves,technical coordination,externalization


In [21]:
data = label_df_1.values
print(type(data))

<class 'numpy.ndarray'>


In [161]:
group = dict(Counter(data[:,0]))
g_k = np.array(list(group))
print("group:",group)
print(g_k)

esp = dict(Counter(data[:,2]))
e_k = np.array(list(esp))
print("epistemic:",esp)
print(e_k)

arg = dict(Counter(data[:,3]))
a_k = np.array(list(arg))
print("argumentation:",arg)
print(a_k)

coo = dict(Counter(data[:,4]))
c_k = np.array(list(coo))
print("coordination:",coo)
print(c_k)

soc = dict(Counter(data[:,5]))
s_k = np.array(list(soc))
print("social:",soc)
print(s_k)


print("---------")

g1 = data[(data[:, 0] == g[45])]


g_name = np.array(list(dict(Counter(g1[:,1]))))
print(g_name)


g_matrix = np.zeros(45).reshape(3,15)
print(g_matrix)
for line in g1:
    for i in range(3):
        if g_name[i] == line[1]:
#             print(i,"--",line)
            if e_k[0] == line[2]:
                g_matrix[i][0] += 1
                g_matrix[i][2] += 1
            if e_k[1] == line[2]:
                g_matrix[i][1] += 1
                g_matrix[i][2] += 1
            if a_k[0] == line[3]:
                g_matrix[i][3] += 1
                g_matrix[i][6] += 1
            if a_k[1] == line[3]:
                g_matrix[i][4] += 1
                g_matrix[i][6] += 1
            if a_k[2] == line[3]:
                g_matrix[i][5] += 1
                g_matrix[i][6] += 1
            if c_k[0] == line[4]:
                g_matrix[i][7] += 1
                g_matrix[i][10] += 1
            if c_k[1] == line[4]:
                g_matrix[i][8] += 1
                g_matrix[i][10] += 1
            if c_k[2] == line[4]:
                g_matrix[i][9] += 1
                g_matrix[i][10] += 1
            if s_k[0] == line[5]:
                g_matrix[i][11] += 1
                g_matrix[i][14] += 1
            if s_k[1] == line[5]:
                g_matrix[i][12] += 1
                g_matrix[i][14] += 1
            if s_k[2] == line[5]:
                g_matrix[i][13] += 1
                g_matrix[i][14] += 1
print(g_matrix)
np.savetxt("bar.csv", g_matrix, delimiter=",", fmt='%i')

for n in g_name:
    print(n)


group: {1: 54, 2: 50, 3: 47, 4: 86, 5: 62, 6: 107, 7: 61, 8: 66, 9: 98, 10: 57, 11: 29, 12: 58, 13: 61, 14: 28, 15: 61, 16: 53, 17: 59, 18: 79, 19: 74, 20: 49, 21: 44, 22: 59, 23: 76, 24: 21, 25: 41, 26: 70, 27: 67, 28: 73, 29: 117, 30: 38, 31: 36, 32: 64, 33: 50, 34: 35, 35: 77, 36: 60, 37: 69, 38: 30, 39: 83, 40: 90, 41: 47, 42: 34, 43: 66, 44: 51, 45: 60, 46: 46}
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
epistemic: {'off task': 1110, 'on task': 1633}
['off task' 'on task']
argumentation: {'non-argumentative moves': 1638, 'simple claim': 1082, 'grounded claim': 23}
['non-argumentative moves' 'simple claim' 'grounded claim']
coordination: {'blank': 2368, 'technical coordination': 360, 'proceedings': 15}
['blank' 'technical coordination' 'proceedings']
social: {'quick consensus building': 421, 'externalization': 2170, 'elicitation': 152}
['quick consensus building' 'externalization' 'elic

In [4]:
# test

# gid = Counter(result_svm_df.values[:,2]).keys()
# print(gid)
# print(type(gid))
# gidd =np.arange(46)+1
# print(gidd)
# print(type(gidd))
# print(type(result_svm_df.values))
# d = result_svm_df.values
# print(d.shape)
# print(d[np.where(d[:,2]==1)])
# xx = d[np.where(d[:,2]==1)]
# print(Counter(xx[:,5]))


In [5]:
# qqq = [np.array(list(Counter(d[np.where(d[:,2]==i)][:,5]).values())) for i in gid]

d = result_svm_df.values
gid = np.arange(46)+1
data = np.array([[0,0]])  
for i in gid:
    x = d[np.where(d[:,2]==i)]
    c = Counter(x[:,5])
    print(c)
    data = np.vstack([data, [[c['on task'],c['off task']]]])
data = data[1:]
print(data)

Counter({'on task': 32, 'off task': 22})
Counter({'on task': 27, 'off task': 23})
Counter({'on task': 26, 'off task': 21})
Counter({'on task': 58, 'off task': 28})
Counter({'on task': 42, 'off task': 20})
Counter({'on task': 65, 'off task': 42})
Counter({'on task': 32, 'off task': 29})
Counter({'on task': 51, 'off task': 15})
Counter({'on task': 56, 'off task': 42})
Counter({'on task': 34, 'off task': 23})
Counter({'off task': 18, 'on task': 11})
Counter({'off task': 32, 'on task': 26})
Counter({'on task': 40, 'off task': 21})
Counter({'off task': 17, 'on task': 11})
Counter({'on task': 38, 'off task': 23})
Counter({'on task': 34, 'off task': 19})
Counter({'on task': 33, 'off task': 26})
Counter({'on task': 53, 'off task': 26})
Counter({'on task': 55, 'off task': 19})
Counter({'on task': 28, 'off task': 21})
Counter({'on task': 26, 'off task': 18})
Counter({'off task': 32, 'on task': 27})
Counter({'on task': 53, 'off task': 23})
Counter({'on task': 15, 'off task': 6})
Counter({'on task

In [12]:
print(type(data))
print(data.shape)

sum = np.sum(data)
ave_on = np.sum(data,axis=0)[0]/46
ave_off = np.sum(data,axis=0)[1]/46

print(sum)
print(ave_on)
print(ave_off)

<class 'numpy.ndarray'>
(46, 2)
2743
35.84782608695652
23.782608695652176


In [15]:
for i in np.arange(46):
    print(data[i][0],data[i][1],data[i][0]+data[i][1])
    print()


32 22 54
27 23 50
26 21 47
58 28 86
42 20 62
65 42 107
32 29 61
51 15 66
56 42 98
34 23 57
11 18 29
26 32 58
40 21 61
11 17 28
38 23 61
34 19 53
33 26 59
53 26 79
55 19 74
28 21 49
26 18 44
27 32 59
53 23 76
15 6 21
21 20 41
42 28 70
38 29 67
47 26 73
61 56 117
17 21 38
20 16 36
43 21 64
36 14 50
18 17 35
50 27 77
31 29 60
47 22 69
14 16 30
69 14 83
50 40 90
23 24 47
16 18 34
33 33 66
30 21 51
42 18 60
28 18 46
